## Importing and setting up

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Datasets/speech_to_text

/content/drive/MyDrive/Datasets/speech_to_text


In [ ]:
!pip install -Uqq torch # transformers

In [ ]:
!pip install -Uqq transformers[sentencepiece] transformers[torch] datasets evaluate ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os
import json

In [ ]:
import torch
import transformers
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [ ]:
!ls

dataloaders			   models
medium_yt_data.csv		   onnx_inference.ipynb
medium_yt_data_processed.csv	   tag_classifier_training.ipynb
med_yt_data_processed_revised.csv  tag_types_encoded.json
med_yt_data_revised.csv


In [ ]:
df = pd.read_csv("medium_yt_data_processed.csv")

In [ ]:
df.head()

,url,text,tags
0,https://medium.com/american-haiku/you-are-never-alone-with-a-dog-17cdb6ce5db2,"Sign up for American Haiku Steamship To Writing History\n\nBy American Haiku\n\nWriting takes practice. American Haiku is a great way to put your words from your fingers to your piece of paper. Don't quit, you can do it. Take a look","['Pets', 'Dogs', 'Haiku', 'Christmas', 'Poem']"
1,https://medium.com/laravel-news/laracon-tickets-are-now-on-sale-81d08ca477d6,"Laracon 2016 is scheduled to be back in Louisville KY, July 27th through 29th. The full schedule has yet to be announced but it will include typical sessions like last year with a special day for workshops. Ticket prices are $425.00 if you reserve early, otherwise general admission will cost $525.00.\n\nConfirmed speakers include Taylor Otwell, Chris Fidao, Adam Wathan, Jack McDade, and Evan You.\n\nIf you’d like to purchase tickets or submit a talk visit the official Laracon site. If you’d like to see some of the highlights of last years event check out the Laracon 2015 Recap.","['Conferences', 'Laravel', 'Laracon']"
2,https://medium.com/@alexisaltieri/i-need-to-vent-and-its-a-long-time-coming-cc310d23eeb2,"I need to vent and it’s a long time coming. I’m tired of supporting someone else’s dream. I’m tired of being treated like less than a human. I’m tired of grinding everyday for someone who could care less if I was gone tomorrow, and I really mean THEY.COULD.CARE.LESS.\n\nI’ve been with the same company we will call it for 4 years.\n\nThe first time I was with this company, I loved every part of it. I loved my co workers , I loved my manager , I loved the atmosphere , the people I interacted with everyday and more. I ended up “leaving” to move around for 2 years with my husband. I worked in ...","['Know Your Worth', 'Foodies', 'Resturants', 'Bartender', 'I Quit']"
3,https://medium.com/bitfolioapp/introducing-bitfolio-3-c9c27ba570c,"It’s been a while since our last development update, but we are back with a bang, Bitfolio 3 for iOS is now available! 🎉\n\nAnd there is more, this is just the beginning.\n\nThe big challenge in developing Bitfolio 3 was having to rewrite part of our infrastructure to accomodate our vision.\n\nGood news, the hard part is mostly done and Bitfolio 3.0.0 is the just the first of many updates.\n\nSo let’s see what’s new and what’s coming!\n\nWhat’s new\n\nEverybody was asking for it and we finally have it: exchage sync support! 🙌\n\nWe’ve also revamped our UI, the main Portfolio screen will no...","['Bitcoin', 'iOS', 'Cryptocurrency', 'Binance', 'Apple']"
4,https://medium.com/@chirag5676/the-incredible-power-of-prayer-to-protect-to-guide-and-to-save-lives-9f8cd85e63ca,"Photo by Ben White on Unsplash\n\nPrayer is the act of communicating with a god or spirit in worship. It is also called a devout petition to God or an object of worship.\n\nPrayer has since the history of mankind been one of the most powerful methods to bring about results. Prayers come in different degrees of intensity and definition. A ‘desperate prayer’ to save a life, perhaps yours or someone else’s, is one type of request. A prayer can also be a simple ‘prayer of gratitude.’ I am the beneficiary of the “incredible” power of prayer. I will describe my experiences so that you may embrac...","['Prayer Times', 'Prayer', 'Spirituality', '7 Day Prayer Miracle', 'Power Of Prayer']"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51456 entries, 0 to 51455
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     51456 non-null  object
 1   text    51456 non-null  object
 2   tags    51456 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
tags_list = df['tags'].to_list()
tag_count = {}

for tags in tags_list:
    tag_list = eval(tags)
    for tag in tag_list:
        if tag in tag_count.keys():
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1
print(f"Number of Tags: {len(tag_count)}")
print(tag_count)

Number of Tags: 33316
{'Pets': 78, 'Dogs': 91, 'Haiku': 105, 'Christmas': 438, 'Poem': 364, 'Conferences': 1, 'Laravel': 57, 'Laracon': 1, 'Know Your Worth': 1, 'Foodies': 55, 'Resturants': 1, 'Bartender': 1, 'I Quit': 1, 'Bitcoin': 1521, 'iOS': 218, 'Cryptocurrency': 1589, 'Binance': 70, 'Apple': 158, 'Prayer Times': 1, 'Prayer': 56, 'Spirituality': 487, '7 Day Prayer Miracle': 2, 'Power Of Prayer': 1, 'Politics': 1323, 'Bernie Sanders': 58, 'Customs Brokers': 1, 'Customs Clearance': 1, 'Companies In India': 2, 'Company': 66, 'Chennai': 1, 'Humanitarian': 58, 'Health': 949, 'Technology': 2215, 'Healthcare': 269, 'Education': 832, 'Life': 1559, 'Nature': 238, 'Stories': 81, 'Travel': 558, 'Writing': 1396, 'Blockchain': 1968, 'Proof Of Work': 10, 'Cryptoeconomics': 5, 'Product Design': 213, 'Usability': 29, 'Design': 778, 'UX Design': 237, 'User Experience': 204, 'Elon Musk': 63, 'Dogecoin': 10, 'Tesla': 71, 'Astontech International': 1, 'Humanity': 129, 'Creativity': 544, 'Human Experi

In [ ]:
# thresh = 30
thresh = 100
rare_tags = [ key for key, value in tag_count.items() if value < thresh]
print(len(rare_tags))
print(rare_tags)

33000
['Pets', 'Dogs', 'Conferences', 'Laravel', 'Laracon', 'Know Your Worth', 'Foodies', 'Resturants', 'Bartender', 'I Quit', 'Binance', 'Prayer Times', 'Prayer', '7 Day Prayer Miracle', 'Power Of Prayer', 'Bernie Sanders', 'Customs Brokers', 'Customs Clearance', 'Companies In India', 'Company', 'Chennai', 'Humanitarian', 'Stories', 'Proof Of Work', 'Cryptoeconomics', 'Usability', 'Elon Musk', 'Dogecoin', 'Tesla', 'Astontech International', 'Human Experience', 'Essay', 'Oracle', 'Texas', 'Silicon Valley', 'Sparkml', 'Apache Spark', 'Classification Algorithms', 'Retail', 'Face Mask', 'Unsupervised Learning', 'Anomaly Detection', 'Performance Evaluation', 'Enjoying Life', 'Whimsy', 'Retirement', 'Partnerships', 'Updates', 'Token', 'Spam', 'Keras', 'New Business', 'Mentorship', 'Introduction', 'Software Testing', 'Bugs', 'Debugging', 'Craig Brain', 'Workmanship', 'Xxxchurch', 'Craig Gross', 'Bias', 'Cognitive Science', 'PHP', 'Médium', 'Quran', 'Muslim', 'Islam', 'Fame', 'Connection', 'P

In [ ]:
tags_list = df['tags'].to_list()
revised_tag_list = []
indices_to_drop = []

for idx, tags in enumerate(tags_list):
    tag_list = eval(tags)
    revised_tags = []

    for tag in tag_list:
        if tag not in rare_tags:
            revised_tags.append(tag)

    if len(revised_tags) == 0:
        indices_to_drop.append(idx)
    else:
        revised_tag_list.append(revised_tags)

df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape


(42282, 3)

In [ ]:
df['revised_tags'] = revised_tag_list

In [ ]:
# df.to_csv("med_yt_data_processed_revised.csv", index=False)

In [ ]:
revised_tags_list = df['revised_tags'].to_list()
revised_tag_count = {}

for tags in revised_tags_list:
    tag_list = tags
    for tag in tag_list:
        if tag in revised_tag_count.keys():
            revised_tag_count[tag] += 1
        else:
            revised_tag_count[tag] = 1

print(f"Number of tags: {len(revised_tag_count)}")
print(revised_tag_count)

Number of tags: 316
{'Haiku': 105, 'Christmas': 438, 'Poem': 364, 'Bitcoin': 1521, 'iOS': 218, 'Cryptocurrency': 1589, 'Apple': 158, 'Spirituality': 487, 'Politics': 1323, 'Health': 949, 'Technology': 2215, 'Healthcare': 269, 'Education': 832, 'Life': 1559, 'Nature': 238, 'Travel': 558, 'Writing': 1396, 'Blockchain': 1968, 'Product Design': 213, 'Design': 778, 'UX Design': 237, 'User Experience': 204, 'Humanity': 129, 'Creativity': 544, 'Machine Learning': 1607, 'Big Data': 268, 'Economy': 110, 'Business': 1002, 'Ecommerce': 158, 'Data Science': 1960, 'Energy': 116, 'Python': 1026, 'Startup': 1328, 'Entrepreneurship': 704, 'Motivation': 249, 'Software Development': 1074, 'Work': 441, 'Communication': 151, 'Social Media': 618, 'Psychology': 574, 'Software Engineering': 318, 'Software': 113, 'Self Improvement': 1127, 'Life Lessons': 1283, 'SEO': 214, 'Sex': 215, 'Marriage': 102, 'Money': 463, 'Defi': 246, 'Growth': 215, 'Social Media Marketing': 126, 'Digital Marketing': 401, 'Instagram'

In [ ]:
encode_tag_types = { key: idx for idx, (key, value) in enumerate(revised_tag_count.items()) }

In [ ]:
labels = list(encode_tag_types.keys())
len(labels), labels[:5]

(316, ['Haiku', 'Christmas', 'Poem', 'Bitcoin', 'iOS'])

In [ ]:
categorical_tag_list = []
revised_tags_list = df['revised_tags'].to_list()

for revised_tags in revised_tags_list:
    categorical_list = [0] * len(encode_tag_types)
    for tags in revised_tags:
        tag_type_index = encode_tag_types[tags]
        categorical_list[tag_type_index] = 1
    categorical_tag_list.append(categorical_list)



In [ ]:
print(labels)

['Haiku', 'Christmas', 'Poem', 'Bitcoin', 'iOS', 'Cryptocurrency', 'Apple', 'Spirituality', 'Politics', 'Health', 'Technology', 'Healthcare', 'Education', 'Life', 'Nature', 'Travel', 'Writing', 'Blockchain', 'Product Design', 'Design', 'UX Design', 'User Experience', 'Humanity', 'Creativity', 'Machine Learning', 'Big Data', 'Economy', 'Business', 'Ecommerce', 'Data Science', 'Energy', 'Python', 'Startup', 'Entrepreneurship', 'Motivation', 'Software Development', 'Work', 'Communication', 'Social Media', 'Psychology', 'Software Engineering', 'Software', 'Self Improvement', 'Life Lessons', 'SEO', 'Sex', 'Marriage', 'Money', 'Defi', 'Growth', 'Social Media Marketing', 'Digital Marketing', 'Instagram', 'Blockchain Technology', 'Ethereum', 'Meditation', 'Poetry', 'Photography', 'Flutter', 'Android', 'Equality', 'Culture', 'Racism', 'Art', 'Tech', 'Marketing', 'Venture Capital', 'Leadership', 'Future', 'UX', 'Finance', 'Fintech', 'Business Strategy', 'Environment', 'College', 'Books', 'Book R

In [ ]:
df['tag_categories'] = categorical_tag_list

In [ ]:
df.head(3)

,url,text,tags,revised_tags,tag_categories
0,https://medium.com/american-haiku/you-are-never-alone-with-a-dog-17cdb6ce5db2,"Sign up for American Haiku Steamship To Writing History\n\nBy American Haiku\n\nWriting takes practice. American Haiku is a great way to put your words from your fingers to your piece of paper. Don't quit, you can do it. Take a look","['Pets', 'Dogs', 'Haiku', 'Christmas', 'Poem']","[Haiku, Christmas, Poem]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1,https://medium.com/bitfolioapp/introducing-bitfolio-3-c9c27ba570c,"It’s been a while since our last development update, but we are back with a bang, Bitfolio 3 for iOS is now available! 🎉\n\nAnd there is more, this is just the beginning.\n\nThe big challenge in developing Bitfolio 3 was having to rewrite part of our infrastructure to accomodate our vision.\n\nGood news, the hard part is mostly done and Bitfolio 3.0.0 is the just the first of many updates.\n\nSo let’s see what’s new and what’s coming!\n\nWhat’s new\n\nEverybody was asking for it and we finally have it: exchage sync support! 🙌\n\nWe’ve also revamped our UI, the main Portfolio screen will no...","['Bitcoin', 'iOS', 'Cryptocurrency', 'Binance', 'Apple']","[Bitcoin, iOS, Cryptocurrency, Apple]","[0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
2,https://medium.com/@chirag5676/the-incredible-power-of-prayer-to-protect-to-guide-and-to-save-lives-9f8cd85e63ca,"Photo by Ben White on Unsplash\n\nPrayer is the act of communicating with a god or spirit in worship. It is also called a devout petition to God or an object of worship.\n\nPrayer has since the history of mankind been one of the most powerful methods to bring about results. Prayers come in different degrees of intensity and definition. A ‘desperate prayer’ to save a life, perhaps yours or someone else’s, is one type of request. A prayer can also be a simple ‘prayer of gratitude.’ I am the beneficiary of the “incredible” power of prayer. I will describe my experiences so that you may embrac...","['Prayer Times', 'Prayer', 'Spirituality', '7 Day Prayer Miracle', 'Power Of Prayer']",[Spirituality],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


## Data split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=22)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(38054, 4228)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head(3)

,url,text,tags,revised_tags,tag_categories
9839,https://medium.com/securedatakit/tell-the-true-story-of-progress-with-sdk-analyze-589b41cd4add,"Part of why we love what we do so much at Secure Data Kit is being able to play a small role in the incredible work our customers are doing around the world. We’ve found a niche in the world of neglected tropical diseases, and some of the largest (and smallest!) global health organizations and nonprofits are using SDK to collect data on how those diseases are impacting individuals in low-resource communities. Secure Data Kit had become the tool of choice to collect data, but we weren’t satisfied.\n\nThat’s why we launched SDK:Analyze, a suite of reporting and integration tools and services...","['Nonprofit', 'Global Health', 'Data Visualization', 'Data Analysis', 'Data Management']","[Data Visualization, Data Analysis]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
19051,https://medium.com/pixel-magazine/we-need-to-talk-about-carrie-519c81a5a4df,"Emily von Hoffmann: How did the idea for your artist exhibition photobook, “Notes From My Therapist,” arise? Can you describe its concept for our readers?\n\nCarrie Thompson: I made Notes From My Therapist from 2012–2015. I was so unhappy. I was starving physically and mentally. I spent my days surviving, not living. I decided to try therapy. Because my mom was a therapist, I’ve always disliked therapy. She made me go as a child. I was very good at convincing them I was just fine. This time I wanted to be open and honest in the hope that I would be able to live more than just survive.\n\nI...","['Travel', 'Photography', 'Mental Health']","[Travel, Photography, Mental Health]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"
1718,https://medium.com/@kate-bredimus/f-i-d-e-l-i-t-y-8bad08005b8c,"“When the moon hits your eye like a big pizza pie\n\nThat’s amore\n\nWhen the room seems to spin like you’ve had too much gin\n\nYou’re in love\n\nKids are born, you are worn, he feels scorned\n\nAs you hide in the closet\n\nLove must wait, ’til a date, then it’s great\n\nBut you’re both still exhausted” Dean Martin (edited)\n\nIt’s time to revisit our infertile couple back in Brooklyn, before an outbreak of children tore through their life like the Norovirus.\n\nTheir love story begins on a dance floor, as all good ones do. But if we’re going to go back farther to the moment where the cou...","['Short Story', 'Parenthood', 'Brooklyn', 'Fertility']",[Short Story],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


## Fastai and Blurr inference

In [ ]:
os.listdir("models")

['multilabel_tags_classifier.pkl']

In [ ]:
model_path = "models/multilabel_tags_classifier.pkl"
learner_inf = load_learner(model_path)

## Evaluation

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

    targets = [ np.asarray(target) for target in test_df['tag_categories'].to_list() ]
    # print(targets)
    outputs = [ np.asarray(pred) for pred in preds ]
    # print(outputs)

    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    print(f"Accuracy = {accuracy}")

    return

In [ ]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
    abstract = row['text']
    labels = learner_inf.blurr_predict(abstract)[0]['class_indices']
    # pred_tasks = [0] * len(encode_task_types)

    # for label in labels:
    #     pred_tasks[encode_task_types[label]] = 1
    preds.append(labels)

  0%|          | 0/4228 [00:00<?, ?it/s]

In [ ]:
# preds

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.42565251779593993
F1 Score (Macro) = 0.3217115058637432
Accuracy = 0.1291390728476821


In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/tag_classifier.onnx',
    input_names=['input_ids'],
    output_names=['outputs'],
    verbose=True,
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
onnx_model_path = 'models/tag_classifier.onnx'
quantized_onnx_model_path = 'models/tag_classifier_quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## ONNX reference

### Normal onnx

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession('models/tag_classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
    abstract = row['text']
    input_ids = tokenizer(abstract)['input_ids'][:512]

    probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
    probs = torch.FloatTensor(probs)

    masks = torch.sigmoid(probs) >= 0.5
    labels = [ class_labels[idx] for idx, mask in enumerate(masks[0]) if mask ]

    pred_tasks = [0] * len(encode_tag_types)

    for label in labels:
        pred_tasks[encode_tag_types[label]] = 1
    preds.append(pred_tasks)

  0%|          | 0/4228 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.42624458163667406
F1 Score (Macro) = 0.3233943398538506
Accuracy = 0.12677388836329234


### Quantiztion

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
class_labels = list(encode_tag_types.keys())

inf_session = rt.InferenceSession('models/tag_classifier_quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []

for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
    abstract = row['text']
    input_ids = tokenizer(abstract)['input_ids'][:512]

    probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
    probs = torch.FloatTensor(probs)

    masks = torch.sigmoid(probs) >= 0.5
    labels = [ class_labels[idx] for idx, mask in enumerate(masks[0]) if mask ]

    pred_tasks = [0] * len(encode_tag_types)

    for label in labels:
        pred_tasks[encode_tag_types[label]] = 1
    preds.append(pred_tasks)

  0%|          | 0/4228 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.4134764701910742
F1 Score (Macro) = 0.3069070931637103
Accuracy = 0.12535477767265846
